# Natural Langauge Processing

The `09` notebooks will cover a variety of NLP topics. This first notebook will cover how to use text in a CSV document with the `IMDB_SAMPLE` dataset, along with training a model. First let's install what we need

In [0]:
!pip install torch torchvision feather-format kornia pyarrow Pillow wandb --upgrade 
!pip install git+https://github.com/fastai/fastai_dev  

# Data Preperation

With fastai 2.0, the text pipeline has been completely reworked. Now we can pre-tokenize our data beforehand to allow for a much faster time getting our data ready.

Let's grab the imports we will be using

In [0]:
from fastai2.basics import *
from fastai2.text.all import *

Let's grab our data and tokenize it

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)

We now have a variety of `tokenize_` functions designed for tokenizing from various sources. We have a `tokenize_folder`, `tokenize_csv`, and `tokenize_df` function. We will use `tokenize_df` here. 

We need to dictate which column in our `csv` contains our text. In our case it is the `text` column

In [0]:
df = pd.read_csv(path/'texts.csv')

In [0]:
df_tok, count = tokenize_df(df, text_cols='text')

In [101]:
df_tok.head()

,label,is_valid,text
0,negative,False,"[xxbos, xxmaj, un, -, bleeping, -, believable, !, xxmaj, meg, xxmaj, ryan, does, n't, even, look, her, usual, pert, lovable, self, in, this, ,, which, normally, makes, me, forgive, her, shallow, ticky, acting, schtick, ., xxmaj, hard, to, believe, she, was, the, producer, on, this, dog, ., xxmaj, plus, xxmaj, kevin, xxmaj, kline, :, what, kind, of, suicide, trip, has, his, career, been, on, ?, xxmaj, whoosh, …, xxmaj, banzai, xxrep, 3, !, xxmaj, finally, this, was, directed, by, the, guy, who, did, xxmaj, big, xxmaj, chill, ?, xxmaj, must, be, a, replay, of, xxmaj, jonestown, -, hollywood,..."
1,positive,False,"[xxbos, xxmaj, this, is, a, extremely, well, -, made, film, ., xxmaj, the, acting, ,, script, and, camera, -, work, are, all, first, -, rate, ., xxmaj, the, music, is, good, ,, too, ,, though, it, is, mostly, early, in, the, film, ,, when, things, are, still, relatively, cheery, ., xxmaj, there, are, no, really, superstars, in, the, cast, ,, though, several, faces, will, be, familiar, ., xxmaj, the, entire, cast, does, an, excellent, job, with, the, script, ., \n\n, xxmaj, but, it, is, hard, to, watch, ,, because, there, is, no, good, end, to, a, situation, like, the, one, ...]"
2,negative,False,"[xxbos, xxmaj, every, once, in, a, long, while, a, movie, will, come, along, that, will, be, so, awful, that, i, feel, compelled, to, warn, people, ., xxmaj, if, i, labor, all, my, days, and, i, can, save, but, one, soul, from, watching, this, movie, ,, how, great, will, be, my, joy, ., \n\n, xxmaj, where, to, begin, my, discussion, of, pain, ., xxmaj, for, starters, ,, there, was, a, musical, montage, every, five, minutes, ., xxmaj, there, was, no, character, development, ., xxmaj, every, character, was, a, stereotype, ., xxmaj, we, had, swearing, guy, ,, fat, guy, who, eats, donuts, ...]"
3,positive,False,"[xxbos, xxmaj, name, just, says, it, all, ., i, watched, this, movie, with, my, dad, when, it, came, out, and, having, served, in, xxmaj, korea, he, had, great, admiration, for, the, man, ., xxmaj, the, disappointing, thing, about, this, film, is, that, it, only, concentrate, on, a, short, period, of, the, man, 's, life, -, interestingly, enough, the, man, 's, entire, life, would, have, made, such, an, epic, bio, -, pic, that, it, is, staggering, to, imagine, the, cost, for, production, ., \n\n, xxmaj, some, posters, elude, to, the, flawed, characteristics, about, the, man, ,, which, are, ..."
4,negative,False,"[xxbos, xxmaj, this, movie, succeeds, at, being, one, of, the, most, unique, movies, you, 've, seen, ., xxmaj, however, this, comes, from, the, fact, that, you, ca, n't, make, heads, or, tails, of, this, mess, ., xxmaj, it, almost, seems, as, a, series, of, challenges, set, up, to, determine, whether, or, not, you, are, willing, to, walk, out, of, the, movie, and, give, up, the, money, you, just, paid, ., xxmaj, if, you, do, n't, want, to, feel, slighted, you, 'll, sit, through, this, horrible, film, and, develop, a, real, sense, of, pity, for, the, actors, involved, ,, they, 've, ...]"


The above will return a tokenized `DataFrame` along with a `Counter`

count now contains a `Counter` that has all of our words (including tokens) in it, along with how often they show up. To find our most common words, we can pass in `n` to `.most_common`

In [38]:
count.most_common(n=5)

[('xxmaj', 24930), ('the', 14467), (',', 11834), ('.', 11735), ('and', 6949)]

From here we can then call `make_vocab` and pass in this `count` to generate our vocabulary! We can also pass in a minimum frequency and a maximum vocabulary too. By default, this is set to 3 and 60,000 respectively

In [0]:
vocab = make_vocab(count)

vocab is now an array of our vocabulary

In [40]:
vocab[100]

'get'

Now we can generate our splits and our `DataBunch` using the `DataBlock` API. We will use just one block for our language model, a `TextBlock` where we pass in our `vocab` and specify it is a language model (`is_lm`), how we want to grab our data (`ColReader`), and how we want to split our data (Randomly with 10% in validation)

In [0]:
imbd_lm = DataBlock(blocks=(TextBlock(vocab, is_lm=True), ),
                    get_x=ColReader('text'),
                    splitter=RandomSplitter(0.1))

Now we can databunch it, passing in our tokenized `DataFrame`, a batch size, and a sequence length. A sequence length is how many words back we will be keeping during our batch. We will use the last 72 words.

In [0]:
dbunch_lm = imdb_lm.databunch(df_tok, bs=64, seq_len=72)

Now let's take a look at it!

In [47]:
dbunch_lm.show_batch(max_n=6)

,text,text_
0,"xxbos i thoroughly enjoyed this film for its humor and pathos . i especially like the way the characters welcomed xxmaj gina 's various suitors . xxmaj with friends ( and family ) like these anyone would feel xxunk and loved . i found the writing witty and natural and the actors made the material come alive . xxbos xxmaj not only does the film 's author , xxmaj steven xxmaj greenstreet","i thoroughly enjoyed this film for its humor and pathos . i especially like the way the characters welcomed xxmaj gina 's various suitors . xxmaj with friends ( and family ) like these anyone would feel xxunk and loved . i found the writing witty and natural and the actors made the material come alive . xxbos xxmaj not only does the film 's author , xxmaj steven xxmaj greenstreet ,"
1,"that time did n't take it any more seriously than modern viewers could . \n\n xxmaj this movie is exactly what the xxunk tend to think of as a ' typical ' silent movie , with it 's archaic moral structure , wooden acting and bad direction . demille shows that he could be a terrible director , with no sense of pacing , camera xxunk , or skill in handling either","time did n't take it any more seriously than modern viewers could . \n\n xxmaj this movie is exactly what the xxunk tend to think of as a ' typical ' silent movie , with it 's archaic moral structure , wooden acting and bad direction . demille shows that he could be a terrible director , with no sense of pacing , camera xxunk , or skill in handling either script"
2,xxmaj grey xxmaj gardens to anyone who enjoys documentaries . xxmaj perhaps someday someone will come along and do a documentary about this documentary - bringing in the rich xxunk ( and xxunk ) of the xxmaj beales and the whole xxunk of xxmaj xxunk society in the 1970 's . xxbos i was looking forward to xxmaj kathryn xxmaj xxunk 's movie with great anticipation after the endless hype and 6,grey xxmaj gardens to anyone who enjoys documentaries . xxmaj perhaps someday someone will come along and do a documentary about this documentary - bringing in the rich xxunk ( and xxunk ) of the xxmaj beales and the whole xxunk of xxmaj xxunk society in the 1970 's . xxbos i was looking forward to xxmaj kathryn xxmaj xxunk 's movie with great anticipation after the endless hype and 6 xxmaj
3,"movie , nothing much except for dialogue between two characters for an hour and a half , it 's likely to bore all but true fans of the xxmaj beatles ; but it 's a fantastic piece of writing and storytelling , and is both informative and touching . xxmaj for those interested in these two musical giants , very quickly you 'll get over the shock of how different the actors",", nothing much except for dialogue between two characters for an hour and a half , it 's likely to bore all but true fans of the xxmaj beatles ; but it 's a fantastic piece of writing and storytelling , and is both informative and touching . xxmaj for those interested in these two musical giants , very quickly you 'll get over the shock of how different the actors look"
4,"and the undead , with our heroes going into the reactor 's lower levels to take out the flesh eating zombies and xxunk the hole forever ! xxmaj pretty cheesy , but i think it was meant to be . xxmaj still , it moves very fast , has xxunk of gruesome effects and really tries to have some style . xxmaj the acting is uneven , but a few good performances","the undead , with our heroes going into the reactor 's lower levels to take out the flesh eating zombies and xxunk the hole forever ! xxmaj pretty cheesy , but i think it was meant to be . xxmaj still , it moves very fast , has xxunk of gruesome effects and really tries to have some style . xxmaj the acting is uneven , but a few good performances shine"
5,but at least violence and killing are ( finally ) seen to have an emotional and psychological impact on those who xxunk and those who merely witness such acts . xxmaj the whole thing is xxunk of a previous age and pre

Great! Now let's begin training

## Training the Language Model

We have a familiary `language_model_learner` here, in which we can pass in our `DataBunch`, our architecture (`AWD_LSTM`), our vocabulary, optimizer, and our metrics. We will use accuracy and Perplexity. To read more on perplexity see [here](https://en.wikipedia.org/wiki/Perplexity)

In [0]:
opt_func = partial(Adam, wd=0.1)

In [0]:
learn = language_model_learner(dbunch_lm, AWD_LSTM, vocab, opt_func=opt_func, metrics=[accuracy, Perplexity()], path=path)

We can then also use Mixed Precision to speed up training

In [0]:
learn = learn.to_fp16(clip=0.1)

Now let's fit! Since we used a pre-trained WikiText103 model, we need to fit for an epoch and then unfreeze the rest of our layers

In [60]:
learn.fit_one_cycle(1, 2e-3, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.693562,4.143623,0.271171,63.030762,00:29


Great! Now let's train the rest. Ideally you'd want to run for 10 epochs or so, but since this is such a small sample, we will fit once as we overfit quite quickly

In [61]:
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.407096,3.987406,0.285446,53.914879,00:37


Now we have a language model. Let's save the encoder away and move onto our classifier

In [0]:
learn.save_encoder('finetuned')

# Classification

Now let's move onto our classifier. The `DataBlock` for this will look *very* similar. We just need to specify two things. We want our `y`'s to be `CategoryBlocks` (categories), and we need to say how we want to get our x and y's. We can do it one of two ways, we can dictate via an `attrgetter` and  `ColumnReader` which column contains our text, or we can create a function

In [0]:
imdb_clas = DataBlock(blocks=(TextBlock(vocab), CategoryBlock),
                      get_x=attrgetter('text'),
                      get_y=ColReader('label'),
                      splitter=RandomSplitter())

We can then `DataBunch` it the exact same, passing in our tokenized `DataFrame`

In [0]:
dbunch = imdb_clas.databunch(df_tok, bs=64, seq_len=72, dl_type=SortedDL)

In [160]:
dbunch.show_batch(max_n=2)

,text,category
0,"xxbos good movie , good music , good background and an acceptable plot . but the main point again as his movies tend to be , the man is the best actor in xxunk and can turn xxunk into gold . xxunk xxunk . this may be his second best performance after xxunk ( others may disagree ) . although other movies are not far behind . one man that will never ever disappoint you . \n\n good movie although i think xxunk was a luxury this movie could have done without . you can see in his movies , others try very hard to reach his heights and act out of their skins . but this man is really something xxunk . \n\n the movie is cool , the music and direction is excellent plot a bit thin but the screen play and dialog again very good . a must watch .",positive
1,"xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk … xxmaj xxunk xxrep 3 ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . w xxrep 3 o xxrep 3 f !",negative


The other method is to use a custom `get_items` function

In [0]:
def _imdb_items(x): return (
    x['text'], x['label'])

In [0]:
imdb_clas = DataBlock(blocks=(TextBlock(vocab), CategoryBlock),
                      get_items = _imdb_items,
                      splitter=RandomSplitter())

We also need to specify that our data is sorted, so we must pass in `SortedDL` as a `dl_type`

In [0]:
dbunch = imdb_clas.databunch(df_tok, bs=64, seq_len=72, dl_type=SortedDL)

In [172]:
dbunch.show_batch(max_n=4)

Now let's train!

## Training the Classifier

Let's use our `text_classifier_learner` again, and pass in our databunch, the architecture, vocab, our metrics, our dropout multiplier, and our optimizer. We also need to pass in our **original** path to grab our encoder

In [0]:
learn = text_classifier_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy],
                                drop_mult=0.5, opt_func=opt_func, path=path)

We can then load in our encoder from earlier

In [0]:
learn = learn.load_encoder('finetuned');

And lastly convert it over to Mixed Precision

In [0]:
learn = learn.to_fp16(clip=0.1)

And now we can gradually unfreeze for differential learning rates

In [0]:
lr = 1e-1 * 64/128
moms = (0.8,0.7,0.8)
wd = 0.1

In [183]:
learn.fit_one_cycle(1, lr_max=lr, moms=moms, wd=wd)

epoch,train_loss,valid_loss,accuracy,time
0,0.878782,0.870123,0.525000,00:13


In [184]:
learn.freeze_to(-2)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=moms, wd=wd)

epoch,train_loss,valid_loss,accuracy,time
0,0.620861,0.736504,0.715000,00:17


In [185]:
learn.freeze_to(-3)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=moms, wd=wd)

epoch,train_loss,valid_loss,accuracy,time
0,0.475432,0.383862,0.815000,00:30


In [186]:
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(2, slice(lr/(2.6**4), lr), moms=moms, wd=wd)

epoch,train_loss,valid_loss,accuracy,time
0,0.386622,0.374377,0.835000,00:35
1,0.341903,0.388184,0.830000,00:33


This model is not the most accurate when compared to regular `IMDB` as it is a subsample. In `09b` I will show that process through and explain how the API may be different there. Thanks for reading!